### Notes

Note, this requires using PyTorch v0.3.1.  Somewhere between 0.3.1 and 0.4.0 parts of the backend were significantly rewritten, preventing us from performing the following hacks. (Likely has to do with them fusing Variable and Tensor).  That may change once their new API stabilizes.

The nice thing about how this is working is that it should be general enough to work for compute, tree, and federated modes of Grid, depending on how the `receive` function works under the hood.

In [1]:
from grid.clients.torch import TorchClient

/Users/jasonmancuso/anaconda/envs/openmined/lib/python3.6/site-packages/h5py-2.7.1-py3.6-macosx-10.7-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
client = TorchClient(verbose = False)


UPDATE: Connecting to IPFS... this can take a few seconds...

SUCCESS: Connected!!! - My ID: QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4

UPDATE: Querying known workers...
	WORKER: /p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX...FAIL!!!
	WORKER: /p2p-circuit/ipfs/Qmaosc64H6Y29VFCFYJzJXCX9AuRp7RCsekLmajHNVEARD...SUCCESS!!!

UPDATE: Searching for IPFS nodes - 90 found overall - 1 are OpenMined workers          

SUCCESS: Found 1 OpenMined nodes!!!



In [3]:
service_self = client.services['torch_service']

In [4]:
# service_self = client.services['torch_service']
# def hook_float_tensor___init__(service_self):
#     torch.FloatTensor.old___init__ = torch.FloatTensor.__init__
#     def new___init__(self, *args, **kwargs):
#         self.old___init__(*args, **kwargs)
#         self = service_self.register_object(self,False)

#     torch.FloatTensor.__init__ = new___init__

In [5]:
import torch
import inspect
from torch.autograd import Variable
import random
import re
from functools import wraps, partial, partialmethod
from types import *
import imp
# from contextlib import contextmanager

In [6]:
def assign_workers_function(worker_ids):
    def decorate(func):
        @wraps(func)
        def send_to_workers(*args, **kwargs):
            part = func(*args, **kwargs)
            command = compile_command(part)
            for worker in worker_ids:
                print("Placeholder print for sending command to worker {}".format(worker))
                args, kwargs = send_command(command)
            receive_commands(worker_ids)  ## Probably needs to happen async
            return args, kwargs
        return send_to_workers
    return decorate

In [7]:
tensor_types = [torch.FloatTensor,
                torch.DoubleTensor,
                torch.HalfTensor,
                torch.ByteTensor,
                torch.CharTensor,
                torch.ShortTensor,
                torch.IntTensor,
                torch.LongTensor]

In [8]:
def assign_workers_method(worker_ids):
    def decorate(method):
        @wraps(method)
        def send_to_workers(self, *args, **kwargs):
            part = method(self, *args, **kwargs)
            if self.is_pointer_to_remote:
                command = compile_command(part)
                for worker in worker_ids:
                    print("Placeholder print for sending command to worker {}".format(worker))
                    args, kwargs = send_command(command)
                receive_commands(worker_ids)  ## Probably needs to happen async
                return args, kwargs
            else:
                result = part.func(self, *args, **kwargs)
                if type(result) in tensor_types:
                    my_service = self.worker.services['torch_service']
                    result = my_service.register_object(result, False)
                return result
        return send_to_workers
    return decorate

In [9]:
# # Slightly modified to remove parent class dependency
# torch.FloatTensor.old___init__ = torch.FloatTensor.__init__
# def hook_float_tensor___init__():
#     def new___init__(self, tensor, owner=client.services['torch_service'], *args, **kwargs):
#         super(torch.FloatTensor, self).__init__(*args, **kwargs)
#         self = owner.register_object(self, False)

#     torch.FloatTensor.__init__ = new___init__

In [10]:
# service_self = client.services['torch_service']
# def hook_float_tensor___init__(service_self):
#     def new___init__(self, *args, **kwargs):
#         super(torch.FloatTensor, self).__init__(*args, **kwargs)
#         self = service_self.register_object(self,False)

#     torch.FloatTensor.__init__ = new___init__

In [11]:
# def assign_workers_factory(worker_ids):
#     def decorate(method):
#         @wraps(method)
#         def send_to_workers(self, *args, **kwargs):
#             part = method(self, *args, **kwargs)
#             command = compile_command(part)
#             for worker in worker_ids:
#                 print("Placeholder print for sending command to worker {}".format(worker))
#                 args, kwargs = send_command(command)
#             receive_commands(worker_ids)  ## Probably needs to happen async
#             return old_init(*args, **kwargs)
#         return send_to_workers
#     return decorate

In [12]:
def pass_func_args(func):
    @wraps(func)
    def pass_args(*args, **kwargs):
        return partial(func, *args, **kwargs)
    return pass_args

def pass_method_args(method):
    @wraps(method)
    def pass_args(*args, **kwargs):
        return partialmethod(method, *args, **kwargs)
    return pass_args

In [13]:
def send_command(command):
    print(command['command'])
    print([type(arg) for arg in command['args']])
    print([type(pair) for pair in command['kwargs']])
    print('===========')
    print()
    return command['args'], command['kwargs']

def receive_commands(worker_ids):
    print('Placeholder print for receiving commands from workers in the following list')
    print(worker_ids)

In [14]:
def compile_command(partial_func):
    func = partial_func.func
    args = partial_func.args
    kwargs = partial_func.keywords
    command = {}
    command['command'] = func.__name__
    command['command_type'] = type(func)
    command['args'] = args
    command['kwargs'] = kwargs
    command['arg_types'] = [type(x) for x in args]
    command['kwarg_types'] = [type(kwargs[x]) for x in kwargs]
    return command

In [15]:
%%time
for x in range(100000):
    y = torch.FloatTensor([[2,2],[2,2]])
    z = torch.FloatTensor([[1,1],[1,1]])
    res = y.add(z)

CPU times: user 1.3 s, sys: 45.2 ms, total: 1.35 s
Wall time: 1.34 s


In [16]:
%%time

for attr in dir(torch):
    if attr == 'typename':
        continue
    if type(torch.__getattribute__(attr)) in [FunctionType, BuiltinFunctionType]:
        torch.__setattr__(attr, assign_workers_function(['A1','B1', 'B2'])(pass_func_args(torch.__getattribute__(attr))))

exclude = ['ndimension', 'nelement', 'size','numel', 'ser', 'de']
for attr in dir(torch.FloatTensor):
    lit = getattr(torch.FloatTensor, attr)
    is_desc = inspect.ismethoddescriptor(lit)
    is_func = type(lit)==FunctionType
    is_mappingproxy = attr == '__dict__'
    try:
        is_service_func = 'TorchService' in lit.__qualname__
    except:
        is_service_func = False
    is_base = attr in dir(object)
    is_old = re.match('old*', attr) is not None
    if attr in exclude:
        print(attr,' skipped')
        continue
    if (is_desc or (is_func and not is_service_func)) and not is_base and not is_old:
        print(attr)
        setattr(torch.FloatTensor, 'old_{}'.format(attr), lit)
        setattr(torch.FloatTensor, attr, assign_workers_method(['A1','B1', 'B2'])(pass_method_args(lit)))
    else:
        print(attr, ' skipped')

__add__
__and__
__array__
__array_wrap__
__bool__
__class__  skipped
__deepcopy__
__delattr__  skipped
__delitem__
__dict__  skipped
__dir__  skipped
__div__
__doc__  skipped
__eq__  skipped
__float__
__format__  skipped
__ge__  skipped
__getattribute__  skipped
__getitem__
__getstate__
__gt__  skipped
__hash__  skipped
__iadd__
__iand__
__idiv__
__ilshift__
__imul__
__init__  skipped
__init_subclass__  skipped
__int__
__invert__
__ior__
__ipow__
__irshift__
__isub__
__iter__
__itruediv__
__ixor__
__le__  skipped
__len__
__long__
__lshift__
__lt__  skipped
__matmul__
__mod__
__module__  skipped
__mul__
__ne__  skipped
__neg__
__new__  skipped
__nonzero__
__or__
__pow__
__radd__
__rdiv__
__reduce__  skipped
__reduce_ex__  skipped
__repr__  skipped
__rmul__
__rpow__
__rshift__
__rsub__
__rtruediv__
__setattr__  skipped
__setitem__
__setstate__
__sizeof__  skipped
__str__  skipped
__sub__
__subclasshook__  skipped
__truediv__
__weakref__  skipped
__xor__
_advanced_index_add
_advanced_inde

In [17]:
%%time
for x in range(100000):
    y = torch.FloatTensor([[2,2],[2,2]])
    z = torch.FloatTensor([[1,1],[1,1]])
    res = y.add(z)

CPU times: user 1.95 s, sys: 77.1 ms, total: 2.03 s
Wall time: 2.02 s


In [18]:
x = y.add(z)

In [19]:
print(x.is_pointer_to_remote)
print(x.id)

False
6881100897


In [20]:
x


 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [21]:
x.fill_(0)


 0  0
 0  0
[torch.FloatTensor of size 2x2]

In [22]:
print(x)


 0  0
 0  0
[torch.FloatTensor of size 2x2]



In [23]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [24]:
x.normal_()

Placeholder print for sending command to worker A1
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],), {})

In [25]:
x.uniform_()

Placeholder print for sending command to worker A1
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],), {})

In [26]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],
  [ torch.FloatTensor - Location:other_guy ]),
 {})